In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import pandas as pd
import itertools

In [5]:
chunk_size = 100000

filtered_chunks = []

for chunk in pd.read_csv("/content/drive/My Drive/6242Project/Data/US_Accidents_March23.csv", chunksize=chunk_size):
    filtered_chunk = chunk[chunk['State'] == 'GA']
    filtered_chunks.append(filtered_chunk)


df_ga = pd.concat(filtered_chunks, ignore_index=True)

In [6]:
y = df_ga['Severity']
X = df_ga.iloc[:, 11:]
X.drop(columns=['Country', 'Timezone', 'Airport_Code'], inplace = True)
X['Severity'] = y

In [7]:
X['Weather_Timestamp'] = pd.to_datetime(X['Weather_Timestamp'])
X['day_of_week'] = X['Weather_Timestamp'].dt.day_name()


In [8]:
def is_weekday(day):
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    return day in weekdays

# Apply the function to the day_of_week column to create the Weekday column
X['Weekday'] = X['day_of_week'].apply(lambda x: is_weekday(x))

In [9]:
X.drop(columns=['Weather_Timestamp', 'day_of_week'], inplace = True)

In [10]:
X.isna().sum()

Street                     415
City                         0
County                       0
State                        0
Zipcode                      0
Temperature(F)            3158
Wind_Chill(F)            61020
Humidity(%)               3333
Pressure(in)              2567
Visibility(mi)            2967
Wind_Direction            3042
Wind_Speed(mph)          16133
Precipitation(in)        63998
Weather_Condition         2705
Amenity                      0
Bump                         0
Crossing                     0
Give_Way                     0
Junction                     0
No_Exit                      0
Railway                      0
Roundabout                   0
Station                      0
Stop                         0
Traffic_Calming              0
Traffic_Signal               0
Turning_Loop                 0
Sunrise_Sunset             159
Civil_Twilight             159
Nautical_Twilight          159
Astronomical_Twilight      159
Severity                     0
Weekday 

In [11]:
X.head()

,Street,City,County,State,Zipcode,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity,Weekday
0,Senoia Rd,Fairburn,Fulton,GA,30213,63.0,NaN,97.0,29.75,3.0,...,False,False,False,False,Day,Day,Day,Day,3,True
1,GA-402 E,Atlanta,Fulton,GA,30336,63.0,NaN,90.0,29.73,3.0,...,False,False,False,False,Day,Day,Day,Day,3,True
2,Donald Lee Hollowell Pkwy NW,Atlanta,Fulton,GA,30331,63.0,NaN,90.0,29.73,2.5,...,False,False,False,False,Day,Day,Day,Day,3,True
3,Sylvan Rd,Atlanta,Fulton,GA,30344,63.0,NaN,97.0,29.77,9.0,...,False,False,False,False,Day,Day,Day,Day,2,True
4,Arthur Langford Pkwy E,Atlanta,Fulton,GA,30315,63.0,NaN,97.0,29.70,10.0,...,False,False,False,False,Day,Day,Day,Day,3,True


### CREATE FINAL DATAFRAMES

In [12]:
#First DF (Aggregated with missing Zipcodes)
result = X.groupby(['Street', 'City', 'Sunrise_Sunset', 'Weekday','Zipcode']).agg({'Severity': ['mean', 'count']})
print(result)


                                                                        Severity  \
                                                                            mean   
Street                    City       Sunrise_Sunset Weekday Zipcode                
 10th Ave                 Columbus   Day            True    31901       2.000000   
                                     Night          True    31901       2.000000   
 10th St NE               Atlanta    Day            False   30309-4007  2.000000   
                                                    True    30309-3961  2.000000   
                                     Night          True    30309-3961  2.000000   
...                                                                          ...   
Zell Miller Mountain Pkwy Blue Ridge Night          True    30513       4.000000   
                          Cherry Log Day            False   30522       3.000000   
                                     Night          False   30522       2.66

In [25]:
#Second DF (Has all Zipcodes in GA)

#******************** SOURCE MISSING ZIP CODES*************************************
allzips = pd.read_csv("/content/drive/My Drive/6242Project/Data/allZipCodesGA.csv")
# Step 1: Find the set difference of zip codes
missing_zipcodes = set(allzips['zip']) - set(X['Zipcode'])
# Step 2: Filter rows in allzips corresponding to missing zip codes
missing_zipcodes_df = allzips[allzips['zip'].isin(missing_zipcodes)]
# Step 3: Rename columns in missing_zipcodes_df
missing_zipcodes_df.rename(columns={'zip': 'Zipcode','city':'City'}, inplace=True)
missing_zipcodes_df['Street'] =missing_zipcodes_df['City']
#Step 4: Take Unique Missing zipcodes
missing_zipcodes_df.drop_duplicates(subset=['Zipcode'])

#*********ADD DUMMY VALUES FOR MISSING FIELDS FOR ZIPCODES SOURCED ABOVE*****************
# Generate all combinations of "Sunrise_Sunset" and "Weekday"
combinations = list(itertools.product(['Day', 'Night'], [True, False]))
# Create a DataFrame with all combinations for each unique zip code
exploded_rows = []
for zipcode in missing_zipcodes_df['Zipcode'].unique():
    for combo in combinations:
        exploded_rows.append({'Zipcode': zipcode, 'Sunrise_Sunset': combo[0], 'Weekday': combo[1]})
allzipsdf = pd.DataFrame(exploded_rows)
#Add Remaining Columns with Constant Values
allzipsdf['Severity'] = 1
allzipsdf = allzipsdf.merge(missing_zipcodes_df[['Zipcode', 'City', 'Street']], on='Zipcode', how='left')

X1 = pd.concat([X, allzipsdf], ignore_index=True)
result2 = X1.groupby(['Street', 'City', 'Sunrise_Sunset', 'Weekday','Zipcode']).agg({'Severity': ['mean', 'count']})
print(result2)

                                                                        Severity  \
                                                                            mean   
Street                    City       Sunrise_Sunset Weekday Zipcode                
 10th Ave                 Columbus   Day            True    31901       2.000000   
                                     Night          True    31901       2.000000   
 10th St NE               Atlanta    Day            False   30309-4007  2.000000   
                                                    True    30309-3961  2.000000   
                                     Night          True    30309-3961  2.000000   
...                                                                          ...   
Zell Miller Mountain Pkwy Blue Ridge Night          True    30513       4.000000   
                          Cherry Log Day            False   30522       3.000000   
                                     Night          False   30522       2.66

In [27]:
def categorize_temperature(temp):
    if temp < 45:
        return 'Low'
    elif temp > 85:
        return 'High'
    else:
        return 'Moderate'
def categorize_humidity(humid):
    if humid < 30:
        return 'Low'
    elif humid > 50:
        return 'High'
    else:
        return 'Moderate'
def categorize_visibility(vis):
    if vis < 3:
        return 'Low'
    else:
        return 'Moderate'
def categorize_rainfall(rain):
    if pd.isna(rain):
        return 'No'
    else:
        return 'Yes'
# Apply the function to create the new column
X1['Temperature'] = X1['Temperature(F)'].apply(categorize_temperature)
X1['Humidity'] = X1['Humidity(%)'].apply(categorize_humidity)
X1['Visibility'] = X1['Visibility(mi)'].apply(categorize_visibility)
X1['Rainfall'] = X1['Precipitation(in)'].apply(categorize_rainfall)
result3 = X1.groupby(['Street', 'City', 'Sunrise_Sunset', 'Weekday','Zipcode','Temperature','Humidity','Visibility','Rainfall']).agg({'Severity': ['mean', 'count']})
print(result3)

                                                                                                                 Severity  \
                                                                                                                     mean   
Street                    City       Sunrise_Sunset Weekday Zipcode    Temperature Humidity Visibility Rainfall             
 10th Ave                 Columbus   Day            True    31901      High        Moderate Moderate   Yes       2.000000   
                                     Night          True    31901      Moderate    High     Moderate   Yes       2.000000   
 10th St NE               Atlanta    Day            False   30309-4007 Moderate    Moderate Moderate   No        2.000000   
                                                    True    30309-3961 Moderate    High     Moderate   No        2.000000   
                                     Night          True    30309-3961 Moderate    Moderate Moderate   No        2.000000   


In [28]:

#print(result2.dtypes)
result.reset_index(inplace=True)
result2.reset_index(inplace=True)
result3.reset_index(inplace=True)


In [29]:
result3

Street        City Sunrise_Sunset Weekday  \
                                                                      
0                       10th Ave    Columbus            Day    True   
1                       10th Ave    Columbus          Night    True   
2                     10th St NE     Atlanta            Day   False   
3                     10th St NE     Atlanta            Day    True   
4                     10th St NE     Atlanta          Night    True   
...                          ...         ...            ...     ...   
57443  Zell Miller Mountain Pkwy  Cherry Log            Day   False   
57444  Zell Miller Mountain Pkwy  Cherry Log          Night   False   
57445  Zell Miller Mountain Pkwy     Ellijay            Day    True   
57446  Zell Miller Mountain Pkwy     Ellijay            Day    True   
57447  Zell Miller Mountain Pkwy     Ellijay            Day    True   

          Zipcode Temperature  Humidity Visibility Rainfall  Severity        
                                                                 mean count  
0           31901        High  Moderate   Moderate      Yes  2.000000     1  
1           31901    Moderate      High   Moderate      Yes  2.000000     1  
2      30309-4007    Moderate  Moderate   Moderate       No  2.000000     2  
3      30309-3961    Moderate      High   Moderate       No  2.000000     1  
4      30309-3961    Moderate  Moderate   Moderate       No  2.000000     1  
...           ...         ...       ...        ...      ...       ...   ...  
57443       30522    Moderate      High   Moderate      Yes  3.000000     1  
57444       30522    Moderate      High   Moderate      Yes  2.666667     3  
57445       30536    Moderate      High   Moderate       No  4.000000     1  
57446       30536    Moderate  Moderate   Moderate      Yes  2.666667     3  
57447       30540    Moderate  Moderate   Moderate       No  2.000000     1  

[57448 rows x 11 columns]

### Download data for viz

In [30]:
from google.colab import files
csv_string = result3.to_csv(index=False)

# Download the CSV file
with open('safety_scorestemp.csv', 'w') as f:
    f.write(csv_string)
files.download('safety_scorestemp.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>